In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
# import shapely
from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
ontology = pd.read_csv('/home/gk/Downloads/ontology.csv')

In [3]:
ontology.sort_values(by='Специальность').head(15)

,Отрасль,Уровень образования,Специальность,Код направления
21,Добыча нефти и газа,СПО,Геодезист,21.02.14
19,Добыча нефти и газа,СПО,Геодезист,21.02.08
65,Добыча угля и металлических руд,ВО,Геодезист,21.05.01
64,Добыча угля и металлических руд,ВО,Геодезист,21.03.03
58,Добыча угля и металлических руд,СПО,Геодезист,21.02.14
57,Добыча угля и металлических руд,СПО,Геодезист,21.02.13
56,Добыча угля и металлических руд,СПО,Геодезист,21.02.08
20,Добыча нефти и газа,СПО,Геодезист,21.02.13
33,Добыча нефти и газа,ВО,Геодезист,21.05.01
32,Добыча нефти и газа,ВО,Геодезист,21.03.03


In [4]:
vacancies = pd.read_csv("vacancy_unique_cities.csv")
cv = pd.read_csv("curricula_vitae_unique.csv")
responses = pd.read_csv("responses.csv", sep=";")

/tmp/ipykernel_143724/463337616.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  responses = pd.read_csv("responses.csv", sep=";")


In [5]:
display(cv.sort_values(by='position_name').head())
display(vacancies.head())
display(responses.head())

,Unnamed: 0,id_candidate,id_cv,birthday,skills,additional_skills,busy_type,country,date_creation,date_publish,date_modify_inner_info,education_type,experience,gender,industry_code,other_info,position_name,profession_code,relocation,salary
3731281,3731281,196c0730-7bc6-11e6-a940-4376a32b3f45,9dad9a46-7bc6-11e6-a940-4376a32b3f45,1994.0,NaN,NaN,Полная занятость,Российская Федерация,2017-08-11,2016-09-16,2019-05-16 00:00:00,Среднее,0.0,Женский,Safety,NaN,( военная служба ) контролер,NaN,0.0,25000.0
5042454,5042454,214c5860-0ca8-11e7-9db8-ef76bd2a03c1,a6608ea9-0cb1-11e7-9db8-ef76bd2a03c1,1980.0,NaN,"ответственность, внимательность, аккуратность,...",Полная занятость,Российская Федерация,2017-03-19,2017-04-10,2019-05-17 00:00:00,Среднее профессиональное,12.0,Мужской,StateServices,NaN,"( для частных компаний инвестирование ,привлеч...",277288.0,1.0,100000.0
95086,95086,c2c47e00-871d-11ea-9d9c-79a78a7da306,487de730-8722-11ea-9d9c-79a78a7da306,1978.0,"стресоустойчивость, легко обучаема",ответственность,Полная занятость,Российская Федерация,2020-04-25,2020-04-27,2020-04-27,Среднее,5.0,Женский,RootLightIndustry,NaN,( раскройщик ) скорняк,NaN,0.0,20000.0
3405870,3405870,b5b15b30-8479-11ea-b406-3bfa22f2d66b,f39d87e0-847c-11ea-b406-3bfa22f2d66b,1992.0,NaN,NaN,Полная занятость,Российская Федерация,2020-04-22,2020-04-24,2020-04-24,NaN,0.0,Мужской,MechanicalEngineering,NaN,"(covid-19) любая, нужно кормить семью",NaN,0.0,30000.0
5209814,5209814,768c9f10-c13a-11e6-8891-ef76bd2a03c1,51b1b549-c13f-11e6-8891-ef76bd2a03c1,1975.0,Прекрасно разбираюсь в электрооборудовании суд...,Трудолюбив и всегда стремлюсь к повышению проф...,Удаленная,Российская Федерация,2016-12-13,2020-04-09,2020-04-09 00:00:00,Среднее профессиональное,5.0,Мужской,Transport,NaN,(iii/7)конвенционный квалификационный судовой ...,NaN,1.0,80000.0


,Unnamed: 0,identifier,title,id_hiring_organization,industry,profession,region,job_location_address,job_location_geo_longitude,job_location_geo_latitude,okso_code,date_creation,date_posted,date_modify_inner_info,city
0,0,000001d0-1ac6-11ec-a0f6-5d150a183f37,Заместитель директора магазина,1080273002325,Sales,271506.0,200000000000,"Республика Башкортостан, г. Уфа, Первомайская ...",56.06,54.82,NaN,2021-09-21,2021-09-21,2021-09-21,Уфа
1,1,000002a2-2cba-11e8-8475-bf2cfe8c828d,Слесарь-сантехник,310236513700025,Industry,185601.0,2300000000000,Краснодарский край,38.76,44.30,NaN,2018-03-21,2018-03-20,2018-03-31,село Лермонтово
2,2,00000833-2116-11e9-940e-bf2cfe8c828d,Бухгалтер,1027601117400,Finances,203369.0,7600000000000,Ярославская область,38.81,58.06,NaN,2019-01-26,2019-01-25,2019-02-01,Рыбинск
3,3,00000dd2-e96e-11eb-9dd6-bf2cfe8c828d,Кондитер,1173256002941,Food,129014.0,3200000000000,Брянская область,31.58,53.00,NaN,2021-07-20,2021-07-16,2021-07-23,Красногорский район
4,4,00000f42-92c4-11e9-8ba1-bf2cfe8c828d,Водитель автомобиля,1023600605753,Transport,114428.0,3600000000000,Воронежская область,42.08,51.37,NaN,2019-06-19,2019-06-18,2019-06-20,Борисоглебск


,activity_flag_candidate,activity_flag_manager,date_creation,date_creation_mistake,date_last_updated,date_modify,date_modify_mistake,id_candidate,id_cv,id_hiring_organization,is_new,id_reply,id_response,id_vacancy,region_code,response_type
0,1,1,2015-09-29,0,2021-11-01,NaN,NaN,8ec68050-4d0f-11e5-8037-833b590698f7,b783dadc-51f4-11e5-8037-833b590698f7,1125476174625,1,NaN,f52e2660-6699-11e5-9ba8-239645b044d5,a8c6c9e0-621a-11e5-9ba8-239645b044d5,5400000000000,Приглашение
1,1,1,2015-11-05,0,2021-11-01,NaN,NaN,e43f4aa0-839f-11e5-8d54-833b590698f7,58090e70-83a1-11e5-8d54-833b590698f7,311723202400090,1,NaN,aa419ec0-83b3-11e5-8d54-833b590698f7,4d179726-6114-11e5-a426-9370902ea73e,7200000000000,Приглашение
2,1,1,2016-01-04,0,2021-11-01,NaN,NaN,1aeb8810-b2e0-11e5-9455-833b590698f7,29d40b2a-b2e7-11e5-9455-833b590698f7,1153025006254,1,NaN,8ec3fda0-b2fc-11e5-9b39-0163a9ae3d01,bae4bbd0-a574-11e5-b24a-833b590698f7,3000000000000,Приглашение
3,1,1,2016-01-22,0,2021-11-01,NaN,NaN,c94243f0-45f7-11e5-b939-9370902ea73e,1a1e4f57-45fb-11e5-b939-9370902ea73e,5117746053134,1,NaN,24ec4800-c08b-11e5-9eda-0163a9ae3d01,1ac243a6-bf61-11e5-9eda-0163a9ae3d01,7700000000000,Приглашение
4,1,1,2016-02-01,0,2021-11-01,NaN,NaN,3bfa24d0-c6a9-11e5-b178-833b590698f7,a781f4cc-c6aa-11e5-b178-833b590698f7,780204893183,1,NaN,c134b8f0-c88d-11e5-b178-833b590698f7,3f782917-b9a5-11e5-924d-239645b044d5,2500000000000,Приглашение


In [6]:
responses = responses[["date_creation", "id_candidate", "id_cv", "id_hiring_organization", "id_response", "id_vacancy", "region_code", "response_type"]]
vacancies = vacancies.rename(columns={"job_location_geo_latitude": "x", "job_location_geo_longitude": "y"})

cv_cities = pd.read_csv("curricula_vitae_cities.csv")
cv_years = pd.read_csv("curricula_vitae_years.csv")
kladr_2021 = pd.read_csv("kladr.csv")

kladr_2021 = kladr_2021[
    (kladr_2021["region_name"] != "Москва") & 
    (kladr_2021["region_name"] != "Санкт-Петербург") & 
    (kladr_2021["region_name"] != "Севастополь")
    ]

regions = list(kladr_2021["region_name"].unique())
cv_cities = cv_cities[~cv_cities["city"].isin(regions)]

responses["year"] = responses["date_creation"].apply(lambda x: int(float(x.split("-")[0])))
cv_years = cv_years.join(cv_cities.set_index("kladr")[["city", "x", "y"]], on="locality")
cv_years = cv_years[cv_years["city"].notna()]

In [7]:
display(cv_cities.head())
display(cv_years.head())
display(kladr_2021.head())

,Unnamed: 0,kladr,city,place_description,x,y
0,0,7100000100000,Тула,"Тульская область, Тула",54.193122,37.617348
1,1,4800800008500,Телегино,"Липецкая область, Елецкий район, Телегино",52.637483,38.420774
2,2,7200000200000,Тобольск,"Тюменская область, Тобольск",58.200891,68.253995
3,3,800000100000,Элиста,"Республика Калмыкия, Элиста",46.307743,44.269759
4,4,200000500000,Салават,"Республика Башкортостан, Салават",53.361651,55.924681


,id_cv,year,locality,city,x,y
0,00000ab0-9069-11ea-b0f9-79a78a7da306,2020,7100000100000,Тула,54.193122,37.617348
1,00000e40-e0ef-11e9-a406-ef76bd2a03c1,2019,4800800008500,Телегино,52.637483,38.420774
2,000010d8-c027-11e5-9eda-0163a9ae3d01,2016,7200000200000,Тобольск,58.200891,68.253995
3,000010d8-c027-11e5-9eda-0163a9ae3d01,2017,7200000200000,Тобольск,58.200891,68.253995
4,000010d8-c027-11e5-9eda-0163a9ae3d01,2018,7200000200000,Тобольск,58.200891,68.253995


,Unnamed: 0,level,type,code,name,city_code,city_name,dist_code,dist_name,region_code,region_name
0,0,1,region,100000000000,Республика Адыгея,NaN,NaN,NaN,NaN,100000000000,Республика Адыгея
1,1,3,city,100000100000,Майкоп,1.000001e+11,Майкоп,NaN,NaN,100000000000,Республика Адыгея
2,2,4,settlement,100000100200,Веселый,1.000001e+11,Майкоп,NaN,NaN,100000000000,Республика Адыгея
3,3,4,settlement,100000100300,Гавердовский,1.000001e+11,Майкоп,NaN,NaN,100000000000,Республика Адыгея
4,4,4,settlement,100000100400,Подгорный,1.000001e+11,Майкоп,NaN,NaN,100000000000,Республика Адыгея


In [8]:
# join cv_city and vacancy_city

responses = responses.merge(cv_years[['id_cv','year', "city", "x", "y"]], left_on=['id_cv','year'], right_on=['id_cv','year'])
responses = responses.join(
    vacancies.set_index("identifier")[["city", "x", "y"]], 
    on="id_vacancy", lsuffix="_cv", rsuffix="_vacancy"
    )

In [9]:
responses.sort_values(by='year', ascending=False).head()

,date_creation,id_candidate,id_cv,id_hiring_organization,id_response,id_vacancy,region_code,response_type,year,city_cv,x_cv,y_cv,city_vacancy,x_vacancy,y_vacancy
657708,2021-07-20,89264a50-da57-11eb-9816-7bf9d8e248ac,0ffb15f0-da59-11eb-b2d7-3bfa22f2d66b,1205000032291,f84ae5e0-e947-11eb-91a4-6db06c9eaf56,156e0767-ba04-11eb-82d7-c92c5fcfa4a0,5000000000000,Приглашение,2021,Москва,55.755819,37.617644,Люберцы,55.670000,37.910000
928928,2021-06-15,a37b4250-33b6-11ea-8a28-736ab11edb0c,4ae464b0-33c4-11ea-9906-ef76bd2a03c1,e464ffe0-0824-11e9-9458-736ab11edb0c,5d2dad80-cdc8-11eb-87d7-3bfa22f2d66b,df628446-1585-11e9-b44c-736ab11edb0c,4600000000000,Принятие,2021,Курчатов,51.659518,35.657098,Курчатов,51.660000,35.680000
795519,2021-01-28,fdbfed60-2f25-11e9-85a5-ef76bd2a03c1,b677d230-203a-11eb-a2de-ef76bd2a03c1,1087746413468,b9fd5450-6105-11eb-82c1-ef76bd2a03c1,da801320-0912-11eb-a448-1fdc17069750,4200000000000,Отказ,2021,Междуреченск,53.685199,88.080398,Междуреченск,53.680000,88.050000
694702,2021-06-09,1e715e70-5a28-11eb-bf66-7bf9d8e248ac,a57f7e00-5a33-11eb-ac5c-69632329477c,9c7c15d0-f691-11ea-99b0-037acc02728d,e7e2a740-c8ec-11eb-bcde-3bfa22f2d66b,039e6832-9264-11eb-9dd6-bf2cfe8c828d,7700000000000,Отказ,2021,Москва,55.755819,37.617644,Москва,55.751244,37.618423
109654,2021-08-25,14796550-fd3b-11eb-9414-ef1bf93dfa56,d54276e0-fd3c-11eb-bf72-6db06c9eaf56,3ab405e0-fc28-11eb-a76e-e736a3d3ed84,688f36e0-05ad-11ec-a49b-39fca4e8c23e,0d4fac03-fc3a-11eb-9dd6-bf2cfe8c828d,5000000000000,Приглашение,2021,Талдом,56.730924,37.527938,рабочий посёлок Запрудня,56.560000,37.440000


In [10]:
# responses = responses[responses['year']==2021]

In [11]:
organizations = pd.read_csv("organizations.csv", delimiter=";")
organizations["org_code"] = organizations["inn"].fillna(organizations["ogrn"])

organization_list = responses[responses["city_vacancy"].isna()]["id_hiring_organization"].unique()
org_codes = list(organizations[organizations["id_organization"].isin(organization_list)]["org_code"].astype(int))

/tmp/ipykernel_143724/3101074283.py:1: DtypeWarning: Columns (36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  organizations = pd.read_csv("organizations.csv", delimiter=";")


In [12]:
# from dadata import Dadata
# dadata = Dadata('key')

In [13]:
# resps=[]
# for c, code in enumerate(org_codes[8867:9000]):
#     print(f'{c}/{len(org_codes[8867:9000])}', end="\r")
#     resps.append(dadata.find_by_id(name="party", query=f"{code}"))

In [14]:
# import pickle
# with open("orgs_resp2500.pkl", "wb") as f:
#     pickle.dump(resps3, f)

In [15]:
import pickle
with open("orgs_resp2500.pkl", "rb") as f:
    r1 = pickle.load(f)

with open("orgs_resp3006.pkl", "rb") as f:
    r2 = pickle.load(f)

with open("orgs_resp3494.pkl", "rb") as f:
    r3 = pickle.load(f)

with open("orgs_resp2.pkl", "rb") as f:
    r4 = pickle.load(f)



In [16]:
r = r1 + r2 + r3 + r4

In [43]:
with open("orgs_from_dadata_all.pkl", "wb") as f:
    pickle.dump(r, f)

In [17]:
len(r)

13750

In [18]:
# import pickle
# with open("orgs_resp.pkl", "rb") as f:
#     resps = pickle.load(f)

In [19]:
import pandas as pd
org_cities = pd.DataFrame(r)

In [20]:
org_cities.shape[0]

13750

In [21]:
org_cities = pd.json_normalize(org_cities.iloc[:, 0])

In [22]:
org_cities

,value,unrestricted_value,data.kpp,data.capital,data.invalid,data.management.name,data.management.post,data.management.disqualified,data.founders,data.managers,...,data.fio.surname,data.fio.name,data.fio.patronymic,data.fio.gender,data.fio.source,data.fio.qc,data.finance,data.management,data.opf,data.address
0,"ООО ""НАДЕЖДА""","ООО ""НАДЕЖДА""",525001001,NaN,NaN,Кузнецова Марина Александровна,ДИРЕКТОР,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ИП Богдашкин Эдуард Игоревич,ИП Богдашкин Эдуард Игоревич,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Богдашкин,Эдуард,Игоревич,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"ГБУ ЦДС ""ПАЛЛАДА""","ГБУ ЦДС ""ПАЛЛАДА""",771501001,NaN,NaN,Ерыкина Наталья Николаевна,ДИРЕКТОР,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"ООО ""САТ""","ООО ""САТ""",110601001,NaN,NaN,Клюпа Олег Владимирович,ДИРЕКТОР,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"ООО ""АРСЕНАЛ-НЕВА""","ООО ""АРСЕНАЛ-НЕВА""",781401001,NaN,NaN,Тимофеев Иван Валериевич,Ликвидатор,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13745,"ООО ""ДИЭЛ-М""","ООО ""ДИЭЛ-М""",772901001,NaN,NaN,Петров Артур Владимирович,ГЕНЕРАЛЬНЫЙ ДИРЕКТОР,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13746,"ООО ""СМП 707""","ООО ""СМП 707""",312301001,NaN,NaN,Шибаев Антон Денисович,Конкурсный управляющий,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13747,"ФГУП ""НПО ""ТЕХНОМАШ""","ФГУП ""НПО ""ТЕХНОМАШ""",771501001,NaN,NaN,Кузин Анатолий Иванович,И.О. ГЕНЕРАЛЬНОГО ДИРЕКТОРА,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13748,ИП Герасимова Юлия Александровна,ИП Герасимова Юлия Александровна,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Герасимова,Юлия,Александровна,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
org_cities["org_code"] = org_codes
org_cities["data.address.data.city"] = org_cities["data.address.data.city"].fillna(
    org_cities["data.address.data.settlement"]
)
org_cities = org_cities[org_cities["data.address.data.city"] != 0]
org_cities = org_cities.join(
    organizations.set_index("org_code")["id_organization"], on="org_code", how="inner"
)

In [24]:
org_cities = org_cities[
    [
        "org_code",
        "id_organization",
        "data.address.data.geo_lat",
        "data.address.data.geo_lon",
        "data.address.data.city",
    ]
]

In [25]:
org_cities.rename(columns={"data.address.data.geo_lat": "geo_lat",
                           "data.address.data.geo_lon": "geo_lon",
                           "data.address.data.city": "city"}, inplace=True)

In [26]:
responses = responses.join(org_cities.set_index("id_organization")[["city", "geo_lat", "geo_lon"]], on="id_hiring_organization")

responses["city_vacancy"] = responses["city_vacancy"].fillna(responses["city"])
responses["x_vacancy"] = responses["x_vacancy"].fillna(responses["geo_lat"])
responses["y_vacancy"] = responses["y_vacancy"].fillna(responses["geo_lon"])

In [27]:
responses[responses['city_vacancy']=='посёлок Ермаково']

,date_creation,id_candidate,id_cv,id_hiring_organization,id_response,id_vacancy,region_code,response_type,year,city_cv,x_cv,y_cv,city_vacancy,x_vacancy,y_vacancy,city,geo_lat,geo_lon
152273,2020-06-10,bbae3d10-a462-11ea-b2aa-7bf9d8e248ac,0fb62d70-a6aa-11ea-8ab9-f3e0740405b3,1027601110690,f1fbf900-aae0-11ea-9ea3-69632329477c,f814b1c3-a1c0-11ea-94f4-bf2cfe8c828d,7600000000000,Приглашение,2020,Рыбинск,58.04864,38.855711,посёлок Ермаково,58.01,39.02,NaN,NaN,NaN


In [28]:
responses = responses.dropna(subset='city')

In [29]:
cities_gdf = (
    pd.concat(
        [
            responses[["city_cv", "x_cv", "y_cv", "year"]].rename(
                columns={"city_cv": "city", "x_cv": "x", "y_cv": "y"}
            ),
            responses[["city_vacancy", "x_vacancy", "y_vacancy"]].rename(
                columns={"city_vacancy": "city", "x_vacancy": "x", "y_vacancy": "y"}
            ),
        ]
    )
    .drop_duplicates(subset="city")
    .reset_index(drop=True)
)

In [30]:
cities_gdf = gpd.GeoDataFrame(
    cities_gdf, geometry=gpd.points_from_xy(cities_gdf["y"], cities_gdf["x"])
)

In [31]:
cities_gdf = cities_gdf[~cities_gdf.is_empty].reset_index(drop=True)
cities_gdf

,city,x,y,year,geometry
0,Санкт-Петербург,59.939099,30.315877,2015.0,POINT (30.31588 59.93910)
1,Москва,55.755819,37.617644,2016.0,POINT (37.61764 55.75582)
2,Садовое,45.308045,34.65271,2015.0,POINT (34.65271 45.30804)
3,Сенокосное,45.732935,33.516188,2015.0,POINT (33.51619 45.73293)
4,Краснодар,45.03547,38.975313,2015.0,POINT (38.97531 45.03547)
...,...,...,...,...,...
10630,село Зелёновка,53.51,49.61,NaN,POINT (49.61000 53.51000)
10631,село Терновка,51.68,41.59,NaN,POINT (41.59000 51.68000)
10632,рабочий посёлок Муромцево,56.38,75.25,NaN,POINT (75.25000 56.38000)
10633,село Петровское,52.63,40.25,NaN,POINT (40.25000 52.63000)


In [32]:
# edges = responses[responses['year'].isin(range(2015,2022,1))].groupby(['city_cv', 'city_vacancy']).size().reset_index(name='weight')
edges = responses.groupby(['city_cv', 'city_vacancy']).size().reset_index(name='weight')

In [33]:
edges

,city_cv,city_vacancy,weight
0,1-го отделения совхоза Масловский,Александров,1
1,1-го отделения совхоза Масловский,Семилукский район,1
2,1-го отделения совхоза Победа,Архангельск,1
3,1-е Засеймье,Вологда,1
4,1-е Засеймье,Кваркено,1
...,...,...,...
111852,фабрики имени 1-го Мая,Москва,3
111853,фабрики имени 1-го Мая,Санкт-Петербург,1
111854,фабрики имени 1-го Мая,Туруханск,1
111855,фабрики имени 1-го Мая,Южно-Сахалинск,1


In [34]:
edges = edges[edges["city_cv"].isin(cities_gdf["city"].values)]
edges = edges[edges["city_vacancy"].isin(cities_gdf["city"].values)]
edges.reset_index(drop=True, inplace=False)
edges

,city_cv,city_vacancy,weight
0,1-го отделения совхоза Масловский,Александров,1
1,1-го отделения совхоза Масловский,Семилукский район,1
2,1-го отделения совхоза Победа,Архангельск,1
3,1-е Засеймье,Вологда,1
4,1-е Засеймье,Кваркено,1
...,...,...,...
111852,фабрики имени 1-го Мая,Москва,3
111853,фабрики имени 1-го Мая,Санкт-Петербург,1
111854,фабрики имени 1-го Мая,Туруханск,1
111855,фабрики имени 1-го Мая,Южно-Сахалинск,1


In [35]:
G = nx.DiGraph()

In [36]:
G.add_nodes_from(cities_gdf['city'].values)

In [37]:
for row in edges.itertuples():
    G.add_edge(row.city_cv, row.city_vacancy, weight=row.weight)

In [38]:
for node in tqdm(G.nodes()):
    if node in cities_gdf['city'].values:
        G.nodes[node]['geometry'] = str(cities_gdf.loc[cities_gdf['city']==node, 'geometry'].item())

  0%|          | 0/10635 [00:00<?, ?it/s]

In [39]:
len(list(G.edges(data=True)))

111728

In [40]:
nx.write_graphml(G, 'G_all.graphml')

In [41]:
# df_edges = pd.DataFrame(G.edges(data=True), columns=['city_cv', 'city_vacancy', 'weight'])
# df_edges['weight'] = df_edges['weight'].map(lambda x: x.get('weight'))
# df_agg = df_edges.groupby(['city_cv', 'city_vacancy'], as_index=False)['weight'].sum()